# **K Means Para la clasificación de Genomas**

### **1. Importar librerias**

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
import cufflinks as cf
cf.set_config_file(theme='solar', offline=True)

In [4]:
from IPython.display import HTML
import webbrowser
from string import Template
import json

### **2. Obtener datos iniciales**

In [2]:
df_upper = pd.DataFrame()

# Lista de archivos con los genomas
csv_files = ['AI_files/sims_All_Genomes_13V_0_500.txt', 'AI_files/sims_All_Genomes_13V_500_800.txt', 'AI_files/sims_All_Genomes_13V_800_1300.txt',
             'AI_files/sims_All_Genomes_13V_1300_2000.txt', 'AI_files/sims_All_Genomes_13V_2000_2350.txt', 'AI_files/sims_All_Genomes_13V_2350_3222.txt']

for file in csv_files:
    df_upper = pd.concat([df_upper, pd.read_csv(file, header=None)]) # Lectura y concatenación de los archivos en dataFrame

### **3. Dar formato a la matriz**

#### 3.1 Formatear como matriz triangular superior

Los datos tienen forma de una matriz triangular superior, pero se encuentran corrido a la izquierda, por lo tanto, los datos tenián que ser formateados a la derecha.

In [3]:
for i in range(len(df_upper)):
    if i == 0:  # Omitir la primera fila
        continue

    flip_nucleoids = np.flip(np.array(df_upper.iloc[i]))  # Invertir fila

    list_nucleoids = np.delete(flip_nucleoids, range(i)) # Eliminar elementos NaN
    list_nucleoids = np.flip(list_nucleoids)  # Des-invertir fila

    df_upper.iloc[i] = np.concatenate((np.zeros(i), list_nucleoids), axis=0)  # Devolver fila formateada

#### 3.2 Completar matriz

Debído a que es una matriz de correlación y su forma es la de una matriz triangular superior, utilicé esta misma matriz para completar la parte inferior faltante y apliqué 1's en la diagonal.

In [8]:
df_zeros = pd.DataFrame(np.zeros(3221)) # Crear dataFrame con 0's

df_upper.reset_index(drop=True, inplace=True) # Reindexar dataFrame
df_upper = pd.concat([df_zeros, df_upper], axis=1) # Concatenar columna
df_upper = pd.concat([df_upper, df_zeros.transpose()], ignore_index=True) # Concatenar fila

In [26]:
matrix = np.asarray(df_upper) + np.identity(3222) + np.asarray(df_upper.transpose()) # Sumar matrices

#### 3.3 Exportar matriz

In [27]:
pd.DataFrame(matrix).to_csv('sims_All_Genomes_13V.txt', header=None, index=False) # Generar archivo txt con los datos de la matriz

### **4. Determinar el valor optimo de K**

#### 4.1 Obtener datos

In [6]:
df_genomes = pd.read_csv('sims_All_Genomes_13V.txt', header=None) # Lectura de archivo con la matriz de los genomas
matrix = np.asarray(df_genomes) # Convertir df_genomes a una matriz

Decidí establecer el rango de los grupos de [8 , 73) con una diferencia de 4 grupos entre cada uno. La razón de esta decisión fué porque consideré innesesario iterar con un incremento de 1 grupo ya que la diferencia con respecto a la anterior iteración es insignificante.

In [30]:
k = range(8, 73, 4) # Establecer rango de los grupos

#### 4.2 Graficar Elbow Curve

Para encontrar el número de grupos optimos para mis datos, he utilizado el método del codo para visualizar el comportamiento en cada uno de los grupos totales.

In [ ]:
wcss = []
for i in k:
    kmeans = KMeans(n_clusters=i) #entrenar modelo
    wcss.append(kmeans.fit(matrix).inertia_) # Guadar valor de la suma de cuadrados de los grupos

In [64]:
df_elbow = pd.DataFrame(wcss, k)
df_elbow.iplot(dimensions=(900, 400), xTitle='Número de clusters', yTitle='WCSS', title='Elbow Curve') # Mostrar gráfico

A partir del método del codo representado en la gráfica anterior, decidí seleccionar 12 grupos porque en ese punto la curva se empieza a atenuar y por tanto, expresa la mayor parte de la variabilidad. Aunque, como la gráfica de codo no era tan evidente, utilice el método de silueta para corraborar mi decisión.

#### 4.3 Graficar Silhouette

In [38]:
score = []
for i in k:
    kmeans = KMeans(n_clusters=i) #entrenar modelo
    labels = kmeans.fit(matrix).labels_
    score.append(silhouette_score(matrix, labels, metric='euclidean')) # Guadar valor del coeficiente de silhouette

In [63]:
df_silhouette = pd.DataFrame(score, k)
df_silhouette.iplot(dimensions=(900, 400), xTitle='Número de clusters', yTitle='Score', title='Silhouette') # Mostrar gráfico

Como se muestra en la gráfica de silueta, mi elección es apropiada ya que se ajusta mejor a los centroides.

### **5. Aplicar el método K means a los datos**

In [20]:
clustering = KMeans(n_clusters=12) # entrenar con el número de grupos
clustering.fit(matrix) # Ajustar clusters a los datos

KMeans(n_clusters=12)

In [8]:
df_labels = pd.DataFrame(clustering.labels_) # Obtener etiquetas asignadas a los genomas

In [9]:
df_labels.to_csv('clustering_labels.txt', header=None, index=False) # Generar archivo txt con los valores de las etiquetas

### **6. Crear Json con los datos**

Para construir el archivo .json hicé uso de los dataFrames para simplificar su construcción mediante diccionarios de datos.

#### 6.1 Generar arreglo de las etiquetas ordenadas

In [29]:
df_group = pd.read_csv('AI_files/groupOrderFamilySlop.txt', header=None) # Lectura del archivo con las familias ordenados de genomas

In [30]:
df_n_genomes = df_group.pop(2) # Extraer el número de genomas que componen a cada familia

In [31]:
list_order = []
for n_genomes in range(len(df_n_genomes)):
    for i in range(df_n_genomes[n_genomes]):
        list_order.append(n_genomes)  # Añadir etiqueta de grupo ordenado

#### 6.2 Crear links de los datos

In [32]:
df_labels = pd.read_csv('clustering_labels.txt', header=None) # Lectura de las etiquetas de agrupamiento de Kmeans

In [33]:
df_links = pd.DataFrame({'source': list_order[i], 'target': 72 + int(df_labels.transpose()[i])} for i in range(3222)) # Crear dataFrame con los links

---
**Importante:** 
La razón por la que en la gráfica no se veían todos los links de una familia es que las líneas que los unen se superponen, para arreglarlo, he añadido estos bloques de código para aumentar el grosor de las líneas que unen los nodos.

In [34]:
df_duplicated_count = df_links.value_counts(sort=False) # Obtener cuenta de cada tupla repetida
df_links = df_links.drop_duplicates() # Eliminar duplicados

In [35]:
df_links.reset_index(drop=True, inplace=True) # Resetear index del dataFrame de links

duplicate_count = np.asarray(df_duplicated_count.values) # Convertir recuento de valores repetidos a un array
df_links = pd.concat([df_links, pd.DataFrame({'value': duplicate_count})], axis=1) # Concatenar columna a df_links

---

#### 6.3 Crear nodos de los datos

In [37]:
df_rename = pd.concat([df_group, pd.DataFrame(np.arange(72), columns=['group'])], axis=1) # Asignar columna con el no.grupo al dataFrame con las familias ordenadas

In [38]:
df_rename.rename(columns={0: 'order', 1: 'name'}, inplace=True) # Renombrar columnas

In [39]:
df_clusters = pd.DataFrame({'name': 'clusters_' + str(i), 'group': 71 + i} for i in range(1, 13)) # Crear dataFrame con los clusters y sus no.grupo

#### 6.4 Escribir json

In [40]:
# Construir diccionario de datos con los nodos y linnks
data_dict = {'nodes': df_rename.to_dict(orient='records') + df_clusters.to_dict(orient='records'), 'links':df_links.to_dict(orient='records')}

In [41]:
with open('D3JsomSims.json', 'w') as file:
    json.dump(data_dict, file, indent=4) # Convertir diccionario en un archivo json

### **7. Graficar Resultado**

Debido a mi deseo de utilizar una gráfica interactiva y la carencia de bibliotecas con gráficos utiles para esta situación, opté por utilizar la biblioteca D3.js para crear el gráfico e incrustarlo en mi NoteBook.

#### 7.1 Consultar datos

In [42]:
with open('D3JsomSims.json') as file:
    data = json.load(file)  # Consultar datos del archivo json

#### 7.2 Incrustar js de la gráfica

Seleccioné un arco-diagrama porque los arcos de cada nodo, son utiles para analizar las conexiones entre estos. Además, me pareció más bonita la presentación de los datos con esta gráfica.

La platinlla del arco diagrama fué obtenido de la gelería de gráficos de D3.js: https://d3-graph-gallery.com/graph/arc_template.html. Sin embargo, modifiqué algunas características de esté y lo adapté para utilizar los datos de mi json.

In [43]:
js_text_template = Template('''
// set the dimensions and margins of the graph
  const margin = { top: 0, right: 30, bottom: 150, left: 60 },
    width = 1800 - margin.left - margin.right,
    height = 1000 - margin.top - margin.bottom;

  // append the svg object to the body of the page
  const svg = d3
    .select("#my_dataviz")
    .append("svg")
    .attr("viewBox", [0, 0, width + 90, height + 80])
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

  // Read dummy data
  let data = $python_data;
  // List of node names
  const allNodes = data.nodes.map((d) => d.name);

  // List of groups
  let allGroups = data.nodes.map((d) => d.order);
  allGroups = [...new Set(allGroups)];

  // A color scale for groups:
  const color = d3.scaleOrdinal().domain(allGroups).range(d3.schemeSet3);

  // A linear scale for node size
  const size = d3.scaleLinear().domain([1, 10]).range([0.5, 8]);

  // A linear scale to position the nodes on the X axis
  const x = d3.scalePoint().range([0, width]).domain(allNodes);

  // In my input data, links are provided between nodes -id-, NOT between node names.
  // So I have to do a link between this id and the name
  const idToNode = {};
  data.nodes.forEach(function (n) {
    idToNode[n.group] = n;
  });

  // Add the links
  const links = svg
    .selectAll("mylinks")
    .data(data.links)
    .join("path")
    .attr("d", (d) => {
      start = x(idToNode[d.source].name); // X position of start node on the X axis
      end = x(idToNode[d.target].name); // X position of end node
      return [
        "M",
        start,
        height - 30, // the arc starts at the coordinate x=start, y=height-30 (where the starting node is)
        "A", // This means we're gonna build an elliptical arc
        (start - end) / 2,
        ",", // Next 2 lines are the coordinates of the inflexion point. Height of this point is proportional with start - end distance
        (start - end) / 2,
        0,
        0,
        ",",
        start < end ? 1 : 0,
        end,
        ",",
        height - 30,
      ] // We always want the arc on top. So if end is before start, putting 0 here turn the arc upside down.
        .join(" ");
    })
    .style("fill", "none")
    .style("stroke-opacity", 0.8)
    .attr("stroke", "grey")
    .style("stroke-width", (d) => { return d.value/1.8});

  // Add the circle for the nodes
  const nodes = svg
    .selectAll("mynodes")
    .data(
      data.nodes.sort((a, b) => {
        +b.n - +a.n;
      })
    )
    .join("circle")
    .attr("cx", (d) => x(d.name))
    .attr("cy", height - 30)
    .attr("r", (d) => 8)
    .style("fill", (d) => color(d.group))
    .attr("stroke", "white");

  // And give them a label
  const labels = svg
    .selectAll("mylabels")
    .data(data.nodes)
    .join("text")
    .attr("x", 0)
    .attr("y", 0)
    .text((d) => d.name)
    .style("text-anchor", "end")
    .attr(
      "transform",
      (d) => "translate(" + (x(d.name)) + "," + (height-15) + ")rotate(-90)"
    )
    .style("font-size", 10);
  // Add the highlighting functionality
  nodes
    .on("mouseover", function (event, d) {
      // Highlight the nodes: every node is green except of him
      nodes.style("opacity", 0.2);
      d3.select(this).style("opacity", 1);
      // Highlight the connections
      links
        .style("stroke", (a) =>
          a.source === d.group || a.target === d.group ? color(d.order) : "#b8b8b8"
        )
        .style("stroke-opacity", (a) =>
          a.source === d.group || a.target === d.group ? 1 : 0.2
        )
        .style("stroke-width", (a) =>
          a.source === d.group || a.target === d.group ? (d) => { return d.value/1.8} : 1
        );
      labels.style("font-size", (b) => (b.name === d.name ? 11 : 8));
    })
    .on("mouseout", (d) => {
      nodes.style("opacity", 1);
      links
        .style("stroke", "grey")
        .style("stroke-opacity", 0.8)
        .style("stroke-width", (d) => { return d.value/1.8});
      labels.style("font-size", 10);
    });
//});
''')

In [44]:
html_template = Template('''
<!DOCTYPE html>
<meta charset="utf-8" />

<!-- Cargar libreria d3.js -->
<script src="https://d3js.org/d3.v6.js"></script>

<!-- Crear div donde se mostrará la gráfica -->
<div id="my_dataviz"></div>

<script> $js_text </script>
''')

In [45]:
# Cargar datos al js
js_text = js_text_template.substitute({'python_data': json.dumps(data),
                                       'my_dataviz': 'my_dataviz'})
HTML(html_template.substitute({'js_text': js_text}))  # Cargar js_text al html

#### 7.3 Mostrar en el navegador

Al intentar mostrar la gráfica en VisualCode, solo se muestra el área que requieré el gráfico, pero no el gráfico. Por lo tanto, opté por generar un archivo html para la visualización fuera de VisualCode.

In [46]:
with open("grafica_agrupaciones.html", "w") as file:
   file.write(html_template.substitute({'js_text':js_text})) # Generar archivo html con los datos del json

In [9]:
webbrowser.open_new_tab('grafica_agrupaciones.html') # Abrir archivo html exportado en el navegador

True